In [2]:
#importing necessary libraries
from __future__ import print_function
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
from imageai.Detection import VideoObjectDetection
import os
import sys
from random import randint
from math import ceil, sqrt
import natsort
import pandas as pd
import random
from keras.applications.vgg16 import VGG16
import pickle

Using TensorFlow backend.
/data/anaconda3/envs/roadcross/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/data/anaconda3/envs/roadcross/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/data/anaconda3/envs/roadcross/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/data/anaconda3/envs/roadcross/l

In [3]:
user = 'aws'

if user == 'siddhi':
    path_videos = 'C:/RoadCrossingAssistant/Data/Videos/'
    path_labels_csv = 'C:/RoadCrossingAssistant/Data/labels_framewise_csv.csv'
    path_labels_list = 'C:/RoadCrossingAssistant/Data/labels_framewise_list.pkl'

elif user == 'yagnesh':
    path_videos = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos/'
    path_labels_csv = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels_framewise.csv'
    path_labels_list = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels_framewise.pkl'

elif user == 'aws':
    path_videos = '/data/Data/Videos/'
    path_labels_csv = '/data/Data/labels_framewise_csv.csv'
    path_labels_list = '/data/Data/labels_framewise_list.pkl'

videos = glob.glob(path_videos+'video*.MOV')
videos = natsort.natsorted(videos)

# frame-wise labels array
open_file = open(path_labels_list, "rb")
labels_list = pickle.load(open_file)
open_file.close()

In [4]:
from tensorflow.python.client import device_lib
import tensorflow as tf
tf.test.is_gpu_available()


False

In [5]:
#Perform train-test split(80-24)

x = np.arange(104)
#np.random.seed(42)
indices_test = np.random.choice(x, 24, replace=False)
indices_train = np.delete(x, indices_test, axis=0)

videos_train = [videos[ind] for ind in indices_train]
videos_test = [videos[ind] for ind in indices_test]

labels_train_loaded = [labels_list[ind] for ind in indices_train]
labels_test_loaded = [labels_list[ind] for ind in indices_test]

print('len of videos_train: ', len(videos_train))
print('len of videos_test: ', len(videos_test))
print('len of labels_train_loaded: ', len(labels_train_loaded))
print('len of labels_test_loaded: ', len(labels_test_loaded))


len of videos_train:  80
len of videos_test:  24
len of labels_train_loaded:  80
len of labels_test_loaded:  24


In [6]:
def get_labels_from_video(no_frames, safe_duration_list):
    
    '''
    Get labels for a particular video 

    Parameters:
    no_frames(int) : no of frames in the given video
    safe_duration_list(list) : a list of the type [safe_start1, safe_end1, safe_start2, safe_end2,......]

    Returns:
    list : list with len = no of frames and the value at each index represents safe/unsafe at that frame_no (frame_no starting at 0)
    int : -1 if there is no safe duration in video, 1 otherwise
    '''

    labels = [0]*no_frames
    no_safe_durations = int(len(safe_duration_list)/2)
    if(no_safe_durations == 0):
        return labels,-1 # there is no safe duration in the given video so all labels marked 0
    else:

        for i in range(no_safe_durations):
            safe_start = max(safe_duration_list[i*2] - 1, 0)
            safe_end = min(safe_duration_list[i*2 +1] - 1, no_frames-1)
            labels[safe_start:safe_end+1] = [1]*(safe_end-safe_start+1) # marking the value b/w safe_start and safe_end with 1

    if len(labels) > no_frames: #len of labels cannot be greater than no_frames in video
        raise Exception('Check the labels assigned in CSV file!')
    return labels,1

In [7]:
labels_train = []
labels_test = []

for i in range(len(videos_train)):

    print(videos_train[i])
    cap = cv2.VideoCapture(videos_train[i])
    no_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    l, f = get_labels_from_video(no_frames, labels_train_loaded[i])
    labels_train.extend(l)

for i in range(len(videos_test)):

    print(videos_test[i])
    cap = cv2.VideoCapture(videos_test[i])
    no_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    l, f = get_labels_from_video(no_frames, labels_test_loaded[i])
    labels_test.extend(l)

labels_train = np.array(labels_train)
labels_test = np.array(labels_test)

print("shape of labels_train", labels_train.shape)
print("shape of labels_test", labels_test.shape)

/data/Data/Videos/video1.MOV
/data/Data/Videos/video2.MOV
/data/Data/Videos/video3.MOV
/data/Data/Videos/video5.MOV
/data/Data/Videos/video6.MOV
/data/Data/Videos/video7.MOV
/data/Data/Videos/video8.MOV
/data/Data/Videos/video9.MOV
/data/Data/Videos/video11.MOV
/data/Data/Videos/video12.MOV
/data/Data/Videos/video13.MOV
/data/Data/Videos/video14.MOV
/data/Data/Videos/video17.MOV
/data/Data/Videos/video18.MOV
/data/Data/Videos/video19.MOV
/data/Data/Videos/video20.MOV
/data/Data/Videos/video24.MOV
/data/Data/Videos/video25.MOV
/data/Data/Videos/video26.MOV
/data/Data/Videos/video27.MOV
/data/Data/Videos/video28.MOV
/data/Data/Videos/video29.MOV
/data/Data/Videos/video30.MOV
/data/Data/Videos/video31.MOV
/data/Data/Videos/video34.MOV
/data/Data/Videos/video35.MOV
/data/Data/Videos/video36.MOV
/data/Data/Videos/video37.MOV
/data/Data/Videos/video38.MOV
/data/Data/Videos/video39.MOV
/data/Data/Videos/video40.MOV
/data/Data/Videos/video41.MOV
/data/Data/Videos/video43.MOV
/data/Data/Videos/

In [8]:
#get required frame indices

def get_required_indices(labels):

    ind0 = np.where(labels == 0)[0]
    ind1 = np.where(labels == 1)[0]

    print(ind0.shape, ind1.shape)

    if (len(ind0)/len(ind1)) > 1.2:
        print('reducing the number of unsafe frames\n\n')
        len_ind0 = int(len(ind1)*1.2)
        ind0 = ind0[:len_ind0]

        indices_required = np.concatenate((ind0, ind1))
    
    return indices_required

indices_required_train = get_required_indices(labels_train)
indices_required_test = get_required_indices(labels_test)

labels_train_reduced = labels_train[indices_required_train]
labels_test_reduced = labels_test[indices_required_test]

print("shape on labels_train_reduced", labels_train_reduced.shape)
print("shape of labels_test_reduced", labels_test_reduced.shape)



(13433,) (6787,)
reducing the number of unsafe frames


(4004,) (2296,)
reducing the number of unsafe frames


shape on labels_train_reduced (14931,)
shape of labels_test_reduced (5051,)


In [ ]:

def get_required_frames(videos, indices_required):

    frames = []
    ind = -1

    for i in range(len(videos)):

        print(videos[i])
        cap = cv2.VideoCapture(videos[i])
        no_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        #print(no_frames)
        
        while cap.isOpened() :
            success, frame = cap.read()
            if not success:
                break
            ind = ind + 1
            if ind in indices_required:
                #frames_train.append(frame)
                #print(frame.shape)
                frame_resized = cv2.resize(frame, (360, 640), interpolation = cv2.INTER_AREA)
                #print(frame_resized.shape)
                frames.append(frame_resized)
    return np.array(frames)

frames_train = get_required_frames(videos_train, indices_required_train)
frames_test = get_required_frames(videos_test, indices_required_test)

print("shape of frames_train: ",frames_train.shape)
print("shape of frames_test: ",frames_test.shape)

        


/data/Data/Videos/video1.MOV
/data/Data/Videos/video2.MOV
/data/Data/Videos/video3.MOV
/data/Data/Videos/video5.MOV
/data/Data/Videos/video6.MOV
/data/Data/Videos/video7.MOV
/data/Data/Videos/video8.MOV
/data/Data/Videos/video9.MOV
/data/Data/Videos/video11.MOV
/data/Data/Videos/video12.MOV
/data/Data/Videos/video13.MOV
/data/Data/Videos/video14.MOV
/data/Data/Videos/video17.MOV
/data/Data/Videos/video18.MOV
/data/Data/Videos/video19.MOV
/data/Data/Videos/video20.MOV
/data/Data/Videos/video24.MOV
/data/Data/Videos/video25.MOV
/data/Data/Videos/video26.MOV
/data/Data/Videos/video27.MOV
/data/Data/Videos/video28.MOV
/data/Data/Videos/video29.MOV
/data/Data/Videos/video30.MOV
/data/Data/Videos/video31.MOV
/data/Data/Videos/video34.MOV
/data/Data/Videos/video35.MOV
/data/Data/Videos/video36.MOV
/data/Data/Videos/video37.MOV
/data/Data/Videos/video38.MOV
/data/Data/Videos/video39.MOV
/data/Data/Videos/video40.MOV
/data/Data/Videos/video41.MOV
/data/Data/Videos/video43.MOV
/data/Data/Videos/

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#print(labels_train_reduced)
y_train = to_categorical(labels_train_reduced, 2)
y_test = to_categorical(labels_test_reduced, 2)
#print(y_train)

print(y_train.shape)

datagen = ImageDataGenerator(rescale=1. / 255, brightness_range=[0.4, 1.5], channel_shift_range = 50, validation_split=0.2)

# train_generator = datagen.flow(frames_train,
#                                y_train,
#                                batch_size=16)

# datagen = ImageDataGenerator(rescale=1./255.,validation_split=0.2)

train_generator = datagen.flow(frames_train,
                               y_train,
                               batch_size=16,
                               shuffle=True,
                               subset='training')

valid_generator = datagen.flow(frames_train,
                               y_train,
                               batch_size=16,
                               subset='validation')



In [ ]:
# from matplotlib.pyplot import imread, imshow, subplots, show
# def plot(data_generator):
#     """
#     Plots 4 images generated by an object of the ImageDataGenerator class.
#     """
#     data_generator.fit(images)
#     image_iterator = data_generator.flow(images)
    
#     # Plot the images given by the iterator
#     fig, rows = subplots(nrows=1, ncols=4, figsize=(18,18))
#     for row in rows:
#         row.imshow(image_iterator.next()[0].astype('int'))
#         row.axis('off')
#     show()

# image = imread("C:/RoadCrossingAssistant/Website/assets/img/approach_1.1_frames/fn1.jpg")

# # Creating a dataset which contains just one image.
# images = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

# #imshow(images[0])
# #show()

# data_generator = ImageDataGenerator(channel_shift_range = 50)
# plot(data_generator)

In [ ]:
from tensorflow.keras.layers import Flatten, Dense
import keras
import tensorflow
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers 
from tensorflow.keras.applications import MobileNet

base_model = MobileNet(input_shape = (640, 360, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(2, activation='softmax')(x)

model = tensorflow.keras.models.Model(base_model.input, x)
model.compile(
    loss='binary_crossentropy',
    optimizer=tensorflow.keras.optimizers.Adam(),
    metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(train_generator, shuffle='true', epochs=18, verbose=1, batch_size=16)

In [ ]:
print("Evaluate on test data")
results = model.evaluate(frames_test, y_test, batch_size=16)
print("test loss, test acc:", results)

print("Evaluate on train data")
results = model.evaluate(frames_train, y_train, batch_size=16)
print("train loss, t acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions")
predictions = model.predict(frames_test)
print("predictions shape:", predictions.shape)
print(predictions[:10])




In [ ]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(np.argmax(y_test, axis = 1), np.argmax(predictions, axis = 1))

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

In [ ]:
# x = np.array([0,0,1,1,1])
# vv = to_categorical(x)
# print(vv)
# np.argmax(a, axis = 1) 